In [94]:
import numpy as np
import pandas as pd
from textblob import TextBlob
from nltk.corpus import stopwords
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.externals import joblib
from textblob.classifiers import DecisionTreeClassifier, NaiveBayesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB


en_stopwords = stopwords.words('english')

In [95]:
def train(df, target): # short cut function for this dang thing... rather nice for future use.
    return train_test_split(
        df,      # [[col for col in df if col != target]],
        target,
        test_size = .2,
        random_state = 42
    )

In [96]:
def evaluate_model(algorithm, train_test):
    train_X, test_X,train_y,test_y = train_test
    model = algorithm().fit(train_X, train_y.values.ravel())
    score = model.score(test_X,test_y)
    print (f" accuracy {score}")
    return model, score

In [104]:
def k_fold (df, target, algorithm):
    scores = []
#     features #= df[[col for col in df if col != target]]
#     target #= df[target]
    kf = StratifiedKFold(n_splits = 5, random_state=42)
    
    for train_i, test_i, in kf.split( df,target):
        scores.append(evaluate_model(
            algorithm,
            (features.iloc[train_i], features.iloc[test_i], 
             target.iloc[train_i], target.iloc[test_i]
            ))[1]
        )
    return sum(scores)/ len(scores)

In [79]:
def confusion_(y, predict):
    return pd.DataFrame(data = confusion_matrix(y, predict), 
                       columns= ['not','deadbeats'], 
                       index= ['not', 'deadbeats'])

In [61]:
df = pd.read_csv('new_export_dataframe_.csv')
df.columns = ['drop1', 'drop2', 'drop3','drop4', 'target','features']
df1 = df[['target','features']]

In [62]:
df1['features'].apply( # remuving stopwords. 
    lambda x: ' '.join([w for w in x.lower().split() if w not in en_stopwords]))

# create deadbeats as target.
df1['target_d'] = df1['target'].apply(lambda x: True if x == 'Deadbeats' else False)

# reduce data set to eliminate inbalace
temp = df1.query('target_d == 0').sample(900)
temp_d = df1.query('target_d  == 1')
reduced_set = pd.concat([temp, temp_d])

C:\Users\Ed-Laptop\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [63]:
target = reduced_set['target_d']
features = reduced_set['features']


train_X,test_X,train_y,test_y = train(features, target )

In [64]:
# naive bays
train = pd.concat([train_X, train_y], axis=1)
test = pd.concat([test_X, test_y], axis=1)

tbbays = NaiveBayesClassifier(train.values)
tbbays.accuracy(test.values)

0.7098591549295775

In [67]:
tfidf = TfidfVectorizer()
word_count_vectors = tfidf.fit_transform(reduced_set['features'].apply(
    lambda x: ' '.join([w for w in x.lower().split() if w not in en_stopwords])
).values).todense().tolist()
features = pd.DataFrame(
    data = word_count_vectors,
    columns = tfidf.get_feature_names()
)

In [71]:
train_X,test_X,train_y,test_y = train(features, target )

## random forest

In [72]:
forest = RandomForestClassifier(max_depth=20, n_estimators=100).fit(train_X, train_y.values.ravel())
forest.score(test_X,test_y)

0.6676056338028169

In [106]:
k_forest = k_fold(features, target,RandomForestClassifier) # kfold forest

C:\Users\Ed-Laptop\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy 0.5943661971830986


C:\Users\Ed-Laptop\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy 0.6299435028248588


C:\Users\Ed-Laptop\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy 0.6440677966101694


C:\Users\Ed-Laptop\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy 0.6468926553672316


C:\Users\Ed-Laptop\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy 0.5875706214689266


In [81]:
confusion_(test_y, forest.predict(test_X))

,not,deadbeats
not,151,25
deadbeats,93,86


## Naive bayes

In [107]:
naive = evaluate_model(GaussianNB, train(features, target))[0]
confusion_(test_y, naive.predict(test_X))

 accuracy 0.6112676056338028


,not,deadbeats
not,62,114
deadbeats,24,155


In [108]:
k_fold(features, target, GaussianNB ) # kfold naivebays. 

 accuracy 0.5464788732394367
 accuracy 0.53954802259887
 accuracy 0.5536723163841808
 accuracy 0.5480225988700564
 accuracy 0.5254237288135594


0.5426291079812207

## multinomial Naive

In [109]:
mn_naive = evaluate_model(MultinomialNB, train(features, target))[0]
confusion_(test_y, naive.predict(test_X))

 accuracy 0.6929577464788732


,not,deadbeats
not,62,114
deadbeats,24,155


In [110]:
k_fold(features, target, MultinomialNB ) # kfold naivebays.

 accuracy 0.6366197183098592
 accuracy 0.6299435028248588
 accuracy 0.6610169491525424
 accuracy 0.6412429378531074
 accuracy 0.6073446327683616


0.6352335481817459

## Knearest neighbor

In [111]:
knn = evaluate_model(KNeighborsClassifier,train(features, target))[0]
confusion_(test_y, knn.predict(test_X))

 accuracy 0.5464788732394367


,not,deadbeats
not,138,38
deadbeats,123,56


In [112]:
k_fold(features, target,KNeighborsClassifier)

 accuracy 0.5859154929577465
 accuracy 0.5621468926553672
 accuracy 0.596045197740113
 accuracy 0.5903954802259888
 accuracy 0.519774011299435


0.5708554149757301

## Logistic regresion

In [113]:
log = evaluate_model(LogisticRegression,train(features, target))[0]
confusion_(test_y, log.predict(test_X))

 accuracy 0.7126760563380282


C:\Users\Ed-Laptop\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


,not,deadbeats
not,140,36
deadbeats,66,113


In [114]:
k_fold(features, target,LogisticRegression)

C:\Users\Ed-Laptop\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


 accuracy 0.5971830985915493


C:\Users\Ed-Laptop\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


 accuracy 0.6299435028248588


C:\Users\Ed-Laptop\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

C:\Users\Ed-Laptop\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


 accuracy 0.6694915254237288


C:\Users\Ed-Laptop\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


 accuracy 0.596045197740113


0.6324309700007957